In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("sg-server-write")

In [2]:
# test-with-predictions-Medical-Llama3-8B-pretrained

model_id = 'Medical-Llama3-8B'
model_type = 'pretrained'
model_name = "ruslanmv/Medical-Llama3-8B"

In [3]:
import pandas as pd
import os

p_test_df = pd.read_csv('/kaggle/input/symptom-disease-classification/test_primary.csv')
p_test_df = p_test_df[['text','disease']]
# Perform transformations on p_test_df
p_test_df['text'] = "Patient's query or symptoms : " + p_test_df['text']
p_test_df['disease'] = "Identified Health Problem : " + p_test_df['disease']

# Rename columns
p_test_df = p_test_df.rename(columns={'text': 'input', 'disease': 'output'})

# Add new column 'instruction'
instruction = ("You're an Advanced Medical Chatbot - Chatbot Dr. MCX, so assume yourself as a Healthcare Professional "
               "whose job is to listen to the Patient's query or symptoms and predict the underlying Health Problem the patient is suffering. "
               "Carefully listen to the Patient's query or symptoms and output only the Identified Health Problem in the following format: "
               "Identified Health Problem : [Health Problem Identified by you].")
p_test_df['instruction'] = instruction

p_test_df = p_test_df[['output','input','instruction']]


# Create the directory if it doesn't exist
output_dir = '/kaggle/working/test_dataset'
os.makedirs(output_dir, exist_ok=True)

# Define the file path
output_file = os.path.join(output_dir, 'transformed_test_primary.csv')

# Save the DataFrame to CSV
p_test_df.to_csv(output_file, index=False)

# Display the file path
print("Saved transformed test dataset to:", output_file)

Saved transformed test dataset to: /kaggle/working/test_dataset/transformed_test_primary.csv


In [4]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

import os
os.environ["WANDB_DISABLED"] = "true"


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
] # More models at https://huggingface.co/unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-11 19:21:15.417400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 19:21:15.417518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 19:21:15.547357: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer contexts auto supported!
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Inference Template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Setting the model for inference
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN



# generating response and cleaning it
def get_raw_response(instruction, input_text):
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                f"{instruction}", # instruction
                f"{input_text}", # input
                "", # output - blank for generation!
            )
        ], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=16, use_cache=True)
    raw_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return raw_response[0]

def get_response(instruction, input_text):
    raw_response = get_raw_response(instruction=instruction, input_text=input_text)
    # Extract Response from raw_response and return it as a string
    response = raw_response.split("### Response:\n")[1].strip()
    return response

# demo
instruction = ("You're an Advanced Medical Chatbot - Chatbot Dr. MCX, so assume yourself as a Healthcare Professional "
               "whose job is to listen to the Patient's query or symptoms and predict the underlying Health Problem the patient is suffering. "
               "Carefully listen to the Patient's query or symptoms and output only the Identified Health Problem in the following format: "
               "Identified Health Problem : [Health Problem Identified by you].")
input_text = "Patient's query or symptoms : skin rash, blackheads, scarring"

print(get_response(instruction=instruction, input_text=input_text))



config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Identified Health Problem : Acne

### Explanation:
The patient is suffering from


In [7]:
# Function to generate prediction using get_response function
def generate_prediction(row):
    instruction = row['instruction']
    input_text = row['input']
    prediction = get_response(instruction=instruction, input_text=input_text)
    print(f'{model_type} {model_id} prediction ::--> ', prediction)
    return prediction

# Function to generate prediction using get_response function
def generate_prediction_chunk(chunk, batch_index, i):
    chunk['prediction'] = chunk.apply(generate_prediction, axis=1)
    
    # Create the directory if it doesn't exist
    output_dir = '/kaggle/working/output/'
    os.makedirs(output_dir, exist_ok=True)

    # Define the file path
    output_file = os.path.join(output_dir, f'predictions-{model_id}-{model_type}_batch_{batch_index}_i={i}.csv')
    chunk.to_csv(output_file, index=False)

# Load the test.csv file
test_df = pd.read_csv('/kaggle/working/test_dataset/transformed_test_primary.csv')

# Apply the generate_prediction_chunk function to create the prediction column in batches
batch_size = 100
for i in range(0, len(test_df), batch_size):
    chunk = test_df.iloc[i:i+batch_size]
    generate_prediction_chunk(chunk, i // batch_size, i)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


pretrained Medical-Llama3-8B prediction ::-->  Identified Health Problem : Vertigo

### Explanation:
Vertigo is a condition


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


pretrained Medical-Llama3-8B prediction ::-->  Identified Health Problem : Vertigo

### Explanation:
The patient is suffering from


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


pretrained Medical-Llama3-8B prediction ::-->  Identified Health Problem : Varicose Veins

### Explanation:
Varic


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


pretrained Medical-Llama3-8B prediction ::-->  Identified Health Problem : [Health Problem Identified by you].

### Explanation:
The
pretrained Medical-Llama3-8B prediction ::-->  Identified Health Problem : [Health Problem Identified by you].

### Explanation:
The


/tmp/ipykernel_24/1946869621.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['prediction'] = chunk.apply(generate_prediction, axis=1)


In [8]:
import os
import pandas as pd

# Directory containing the CSV files
directory = '/kaggle/working/output/'

# List of file names to merge
file_names = [
    f"predictions-{model_id}-{model_type}_batch_0_i=0.csv",
    f"predictions-{model_id}-{model_type}_batch_1_i=100.csv",
    f"predictions-{model_id}-{model_type}_batch_2_i=200.csv",
    f"predictions-{model_id}-{model_type}_batch_3_i=300.csv",
    f"predictions-{model_id}-{model_type}_batch_4_i=400.csv",
    f"predictions-{model_id}-{model_type}_batch_5_i=500.csv",
    f"predictions-{model_id}-{model_type}_batch_6_i=600.csv"
]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Merge the CSV files in sequence
for file_name in file_names:
    file_path = os.path.join(directory, file_name)
    df = pd.read_csv(file_path)
    merged_df = pd.concat([merged_df, df], ignore_index=True)
merged_df = merged_df[['output','prediction','input','instruction']]

# Create the directory if it doesn't exist
output_dir = '/kaggle/working/results/'
os.makedirs(output_dir, exist_ok=True)

# Define the file path
output_file = os.path.join(output_dir, f"predictions-{model_id}-{model_type}-output.csv")

# Save the merged DataFrame to a new CSV file
merged_df.to_csv(output_file, index=False)